<a href="https://colab.research.google.com/github/ChetanKnowIt/PML_Notes/blob/main/PML_11_NeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/gdrive')
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import *

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
t_path = '/content/gdrive/MyDrive/Datasets/tinydata.csv'
tinydata = pd.read_csv(t_path, index_col = 0)
dum_tiny = pd.get_dummies(tinydata, drop_first=True)
sgd = SGDClassifier(loss='log_loss', random_state = 2022)
X = dum_tiny.drop("Acceptance_like", axis = 1)
y = dum_tiny['Acceptance_like']

In [ ]:
sgd.fit(X, y)
print(sgd.coef_)
print(sgd.intercept_)

ValueError: ignored

In [ ]:
tst_tiny = np.array([[0.3,0.4]])
y_pred = sgd.predict(tst_tiny)
print(y_pred)
y_pred_prob = sgd.predict_proba(tst_tiny)
print(y_pred_prob)

NotFittedError: ignored

In [ ]:
### MLP

from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(2,), activation="logistic", random_state = 2022)
mlp.fit(X, y)
print(mlp.coef_)
print(mlp.intercept_)
tst_tiny = np.array([[0.3,0.4]])
y_pred = mlp.predict(tst_tiny)
print(y_pred)
y_pred_prob = mlp.predict_proba(tst_tiny)
print(y_pred_prob)

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


AttributeError: ignored

# MLP Pipeline 
### Multi Layer Classifier for Bankruptcy

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

bank_path = '/content/gdrive/MyDrive/Datasets/Bankruptcy/Bankruptcy.csv'
Bankruptcy = pd.read_csv(bank_path)
X = Bankruptcy.drop(['YR','D'], axis=1)
y = Bankruptcy['D']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, 
                                                    random_state=2022, train_size=0.7)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
mm = MinMaxScaler()
mlp = MLPClassifier(activation = 'logistic', random_state =2022)
pipe = Pipeline([('MM',mm),('MLP',mlp)])

In [ ]:
pipe.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('MM', MinMaxScaler()),
                ('MLP',
                 MLPClassifier(activation='logistic', random_state=2022))])

In [ ]:
y_pred = pipe.predict(X_test)
#print(y_pred)
print(accuracy_score(y_test, y_pred))
y_pred_prob = pipe.predict_proba(X_test)[:,1]
#print(y_pred_prob)
print(roc_auc_score(y_test, y_pred_prob))

0.9
0.9425


### Results with MLP:

| **Metric** | **Score** |
|------------|-----------|
| Accuracy   | 0.9       |
| roc_auc    | 0.94      |

In [ ]:
### USING GRID SEARCH CV

pipe = Pipeline([('MM',mm),('MLP',mlp)])
pipe.get_params()

{'memory': None,
 'steps': [('MM', MinMaxScaler()),
  ('MLP', MLPClassifier(activation='logistic', random_state=2022))],
 'verbose': False,
 'MM': MinMaxScaler(),
 'MLP': MLPClassifier(activation='logistic', random_state=2022),
 'MM__clip': False,
 'MM__copy': True,
 'MM__feature_range': (0, 1),
 'MLP__activation': 'logistic',
 'MLP__alpha': 0.0001,
 'MLP__batch_size': 'auto',
 'MLP__beta_1': 0.9,
 'MLP__beta_2': 0.999,
 'MLP__early_stopping': False,
 'MLP__epsilon': 1e-08,
 'MLP__hidden_layer_sizes': (100,),
 'MLP__learning_rate': 'constant',
 'MLP__learning_rate_init': 0.001,
 'MLP__max_fun': 15000,
 'MLP__max_iter': 200,
 'MLP__momentum': 0.9,
 'MLP__n_iter_no_change': 10,
 'MLP__nesterovs_momentum': True,
 'MLP__power_t': 0.5,
 'MLP__random_state': 2022,
 'MLP__shuffle': True,
 'MLP__solver': 'adam',
 'MLP__tol': 0.0001,
 'MLP__validation_fraction': 0.1,
 'MLP__verbose': False,
 'MLP__warm_start': False}

In [ ]:
params = { 'MLP__hidden_layer_sizes': [(20,10,5),(10,5),(50,)],
          'MLP__activation': ('logistic','tanh'),
          'MLP__learning_rate':('constant','invscaling','adaptive'),
          'MLP__learning_rate_init':(0.001,0.3,0.5)
          }

kfold = StratifiedKFold(n_splits =5, shuffle = True, random_state = 2022)
gcv = GridSearchCV(pipe, param_grid = params, cv = kfold, scoring = 'roc_auc', verbose = 3)          
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=0.951 total time=   0.1s
[CV 2/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=0.780 total time=   0.1s
[CV 3/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=1.000 total time=   0.1s
[CV 4/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=0.899 total time=   0.1s
[CV 5/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=0.994 total time=   0.1s
[CV 1/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=con

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 3/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=0.828 total time=   0.1s
[CV 4/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=0.870 total time=   0.1s
[CV 5/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=0.941 total time=   0.1s
[CV 1/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.3;, score=0.044 total time=   0.0s
[CV 2/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.3;, score=0.099 total time=   0.0s
[CV 3/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.3;, score=0.905 total time=   0.0s
[CV 4/5] END MLP__acti

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 4/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(10, 5), MLP__learning_rate=invscaling, MLP__learning_rate_init=0.001;, score=0.870 total time=   0.1s
[CV 5/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(10, 5), MLP__learning_rate=invscaling, MLP__learning_rate_init=0.001;, score=0.941 total time=   0.1s
[CV 1/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(10, 5), MLP__learning_rate=invscaling, MLP__learning_rate_init=0.3;, score=0.044 total time=   0.0s
[CV 2/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(10, 5), MLP__learning_rate=invscaling, MLP__learning_rate_init=0.3;, score=0.099 total time=   0.0s
[CV 3/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(10, 5), MLP__learning_rate=invscaling, MLP__learning_rate_init=0.3;, score=0.905 total time=   0.0s
[CV 4/5] END MLP__activation=logistic, MLP__hidden_layer_sizes=(10, 5), MLP__learning_rate=invscaling, MLP__learning_rate_init=0.3;, score=0.030 total time=   0.0s
[CV 5/5] END

...

[CV 2/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.001;, score=0.901 total time=   0.1s
[CV 3/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.001;, score=0.941 total time=   0.1s


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 4/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.001;, score=1.000 total time=   0.1s
[CV 5/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.001;, score=0.988 total time=   0.1s
[CV 1/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.3;, score=1.000 total time=   0.1s


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.3;, score=0.956 total time=   0.1s
[CV 3/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.3;, score=0.988 total time=   0.1s
[CV 4/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.3;, score=0.988 total time=   0.1s
[CV 5/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.3;, score=0.852 total time=   0.0s
[CV 1/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.5;, score=1.000 total time=   0.1s
[CV 2/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.5;, score=0.973 total time=   0.1s
[CV 3/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(50,)

In [ ]:
params = { 'MLP__hidden_layer_sizes': [(20,10,5),(10,5),(50,)],
          'MLP__activation': ['tanh','logistic','identity'],
          'MLP__learning_rate':['constant','invscaling','adaptive'],
          'MLP__learning_rate_init':[0.001,0.3,0.5]
          }

kfold = StratifiedKFold(n_splits =5, shuffle = True, random_state = 2022)
gcv = GridSearchCV(pipe, param_grid = params, cv = kfold, scoring = 'roc_auc', n_jobs = -1)          
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)

{'MLP__activation': 'tanh', 'MLP__hidden_layer_sizes': (50,), 'MLP__learning_rate': 'constant', 'MLP__learning_rate_init': 0.5}
0.9850380388841927


### MLP with concrete

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.neural_network import MLPRegressor
con_path = '/content/gdrive/MyDrive/Datasets/Concrete_Data.csv'
concrete = pd.read_csv(con_path)
X = concrete.drop('Strength', axis =1 )
y = concrete['Strength']

mm = MinMaxScaler()
mlp = MLPRegressor(random_state =2022)
pipe = Pipeline([('MM',mm),('MLP',mlp)])


params = { 'MLP__hidden_layer_sizes': [(6,4,3),(7,5),(10,)],
          'MLP__learning_rate':['constant','invscaling','adaptive'],
          'MLP__learning_rate_init':[0.001,0.3,0.5]
          }

kfold = KFold(n_splits =5, shuffle = True, random_state = 2022)
gcv = GridSearchCV(pipe, param_grid = params, cv = kfold, scoring = 'r2', n_jobs = -1)          
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)


{'MLP__hidden_layer_sizes': (7, 5), 'MLP__learning_rate': 'constant', 'MLP__learning_rate_init': 0.5}
0.7053590288937727


took 23 seconds

### MLP FOR HR Dataset

In [ ]:
hr_path = '/content/gdrive/MyDrive/Datasets/human-resources-analytics/HR_comma_sep.csv'
hr = pd.read_csv(hr_path)
hr_dum = pd.get_dummies(hr, drop_first=True)
X = hr_dum.drop('left', axis = 1)
y = hr_dum['left']

mm = MinMaxScaler()
mlp = MLPClassifier(random_state =2022)
pipe = Pipeline([('MM',mm),('MLP',mlp)])

params = { 'MLP__hidden_layer_sizes': [(20,10,5),(30,20,10),(40,30,10)],
          'MLP__activation': ['tanh','logistic','identity'],
          'MLP__learning_rate':['constant','invscaling','adaptive'],
          'MLP__learning_rate_init':[0.001,0.3,0.5]
          }

kfold = StratifiedKFold(n_splits =5, shuffle = True, random_state = 2022)
gcv = GridSearchCV(pipe, param_grid = params, cv = kfold, scoring = 'roc_auc', n_jobs = -1)          
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)

{'MLP__activation': 'tanh', 'MLP__hidden_layer_sizes': (40, 30, 10), 'MLP__learning_rate': 'constant', 'MLP__learning_rate_init': 0.001}
0.9846420990748508


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
%%timeit 

def test(n):
    return sum(range(n))

n = 10000
loop = 1000

result = timeit.timeit('test(n)', globals=globals(), number=loop)
print(result / loop)


NameError: ignored